In [1]:
%display latex

In [2]:
import itertools
import functools
import numpy as np

In [3]:
def same(*args):
    for arg in args:
        assert args[0] == arg
    return args[0]

# 1

In [4]:
class GaussQuad:
    def __init__(self, alphas, betas, wf):
        self.n = same(len(alphas), len(betas))
        
        self.wf = wf
    
        J = np.zeros((self.n, self.n))
        for k, alpha, beta in zip(range(self.n), alphas, betas):
            J[k, k] = alpha
            if k-1 >= 0:
                J[k-1, k] = sqrt(beta)
                J[k, k-1] = sqrt(beta)

        x = var("x")
        polys = [0, 1]
        for k, alpha, beta in zip(itertools.count(2), alphas, betas):
            poly = (x - alpha)*polys[k-1] - beta*polys[k-2]
            polys.append(poly)
        self.poly = polys[-1]
        self.poly = self.poly.full_simplify()
        
        # Eigenvectors are returned in normalized form.
        # eig.eigenvectors[vector_dimension,vector]
        eig = np.linalg.eig(J)
        eig_val = eig.eigenvalues
        eig_vec = eig.eigenvectors.T

        self.x = eig_val.tolist()
        v = eig_vec.tolist()

        self.w = []
        for k in range(self.n):
            self.w.append(betas[0]*v[k][0]**2)
    
    @property
    def sx(self):
        return var("x")
    
    @property
    def sf(self):
        return function("f")
    
    @functools.cached_property
    def quad(self):
        quad = 0
        for xv, wv in zip(self.x, self.w):
            quad += wv*(self.sf)(xv)
        return quad
    
    @functools.cached_property
    def poly_norm(self):
        coef = max(self.poly.coefficients(), key=lambda a: a[1])[0]
        poly = self.poly / coef
        return poly
        
    @property
    def rest(self):
        a = var("a")
        b = var("b")
        
        with assuming(a < b):
            rest = (
                diff((self.sf)(self.sx), self.sx, 2*self.n)
                / factorial(2*self.n)
                * integral(self.wf(self.sx)*self.poly_norm**2, self.sx, a, b)
            )
        return rest

In [5]:
def jacobi_values(n, lambda_):
    alphas = []
    betas = []
    
    for k in range(n):
        alphas.append(0)
        
        if k == 0:
            betas.append(
                2**(2*lambda_) * (gamma(lambda_ + 1/2))**2 / gamma(2*lambda_ + 1)
            )
        elif k == 1:
            betas.append(
                4 * (lambda_ + 1/2)**2 / ((2*lambda_ + 1)**2 * (2*lambda_ + 2))
            )
        else:
            betas.append(
                4*k*(k + lambda_ - 1/2)*(k + 2*lambda_ - 1)*(k + lambda_ - 1/2)
                /
                (
                    (2*k - 2 + 2*lambda_)*(2*k + 2*lambda_ - 1)**2*(2*k + 2*lambda_)
                )
            )
    
    w = function("w")
    x = var("x")
    w(x) = (1-x**2)**(lambda_ - 1/2)
    
    return alphas, betas, w

In [6]:
lambda_ = var("l")
jacobi_values(5, lambda_)

([0, 0, 0, 0, 0],
 [2^(2*l)*gamma(l + 1/2)^2/gamma(2*l + 1),
  1/2/(l + 1),
  1/2*(2*l + 1)/((l + 2)*(l + 1)),
  3/2*(l + 1)/((l + 3)*(l + 2)),
  (2*l + 3)/((l + 4)*(l + 3))],
 x |--> (-x^2 + 1)^(l - 1/2))

## d

In [13]:
n = 5
lambda_ = 5/6
gq = GaussQuad(*jacobi_values(n, lambda_))

fv = function("f")
x = var("x")
fv(x) = cos(x)
display(fv)

gq.quad.substitute_function(gq.sf, fv).n()
# gq.quad

x |--> cos(x)

1.46307601360770

In [8]:
fv = function("f")
x = var("x")
fv(x) = (1-x**2)**(1/3) * cos(x)

integral(fv, x, -1, 1).n()

1.4630760178083206

## e

In [33]:
rest = (
    1
    / factorial(2*gq.n)
    * integral(gq.wf(gq.sx)*gq.poly_norm**2, gq.sx, -1, 1)
)
rest.n()

5.20076053048014e-10

## 2